In [27]:
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import librosa
import librosa.display
import sounddevice as sd
import json

'''
y, sr = librosa.load(librosa.util.example_audio_file())
fft_win_len = 32 #ms
frame_stride = 24#ms
n_fft = int(fft_win_len*sr/1000.0)
hop_length = int(n_fft - (frame_stride*sr/1000.0))
print(sr)
print(len(y)/sr)
print(sd.default.samplerate)
sd.play(y, samplerate=sr)
time.sleep(20)
sd.stop()
melspec = librosa.feature.melspectrogram(y=y, sr=sr)
#print('get S from mel...')
#S_from_mel = librosa.feature.inverse.mel_to_stft(melspec, sr=sr)
print('get original stft...')
stft = librosa.core.stft(y)
S = np.abs(stft)
print('grlim start')
grlim = librosa.core.griffinlim(S, momentum=0.99)
#grlim_mel = librosa.core.griffinlim(S_from_mel, momentum=0.99)
audio_from_mel = librosa.feature.inverse.mel_to_audio(melspec, sr=sr)
print('grlim end')
db = librosa.power_to_db(melspec, ref=np.max)
print(len(melspec), len(db))
print(type(melspec), type(S), type(y))
print(np.shape(melspec), np.shape(db), np.shape(stft), np.shape(S), np.shape(grlim), np.shape(y))
librosa.display.specshow(db, x_axis='time',
                         y_axis='mel', sr=sr, 
                         hop_length=hop_length, 
                         fmax=8000)
plt.colorbar(format='%+2.0f dB')
plt.title('Mel-frequency spectrogram')
plt.tight_layout()
plt.show()
sd.play(grlim, samplerate=sr)
time.sleep(20)
sd.stop()
sd.play(audio_from_mel, samplerate=sr)
time.sleep(20)
sd.stop()
print('--end--')
'''

def save_stft_json(S, save_path, ori_sr, stft_sr):
    
    data = {'audio' : S, 'ori_sr' : 44100, 'stft_sr' : 8000}
    with open(save_path, 'w') as f:
        json.dump(data, f)
        f.close()
    return

def load_stft_json(path):
    
    assert os.path.exists(path)
    f = open(path)
    data = json.load(f)
    return data['audio'], data['ori_sr'], data['stft_sr']


def audio_file_to_mel(ori, save_dir, save_name, ori_sr=44100, mel_sr=16000, **kwargs):
    
    assert os.path.exists(ori)
    
    ado, _ = librosa.core.load(ori, ori_sr)
    ado = librosa.core.resample(ado, ori_sr, mel_sr)
    melspec = librosa.feature.melspectrogram(ado, mel_sr, **kwargs)
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    full_name = os.path.join(save_dir, save_name)
    np.save(full_name, melspec)
    return np.shape(melspec)


def audio_file_to_stft(ori, save_dir, save_name, ori_sr=44100, stft_sr=16000, **kwargs):
    
    assert os.path.exists(ori)
    
    ado, _ = librosa.load(ori, ori_sr)
    ado = librosa.core.resample(ado, ori_sr, target_sr)
    S = librosa.core.stft(ado, target_sr, **kwargs)
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    full_name = os.path.join(save_dir, save_name)
    np.save(full_name, S)
    return np.shape(S)


def melspec_to_audio(melspec, ori_sr=44100, mel_sr=16000, **kwargs):
    
    ado = librosa.feature.inverse.mel_to_audio(melspec, sr=mel_sr, **kwargs)
    resampled = librosa.core.resample(ado, mel_sr, ori_sr)
    return resampled


def mel_file_to_audio(ori_path, ori_sr=44100, mel_sr=16000, **kwargs):
    
    assert os.path.exists(ori_path)
    
    melspec = np.load(ori_path)
    return melspec_to_audio(melspec, ori_sr=ori_sr, mel_sr=mel_sr, **kwargs)


def stft_file_to_audio(ori_path, ori_sr=44100, stft_sr=16000, **kwargs):
    
    assert os.path.exists(ori_path)
    
    S = np.load(ori_path)
    ado = librosa.core.istft(S, **kwargs)
    resampled = librosa.core.resample(ado, stft_sr, ori_sr)
    return resampled


audio_file_to_mel('original_mp3/AUDIO_Default_308.mp3', 'mel', '0', mel_sr=8000)
print('start')
ori_mp3, orig_sr = librosa.load('original_mp3/AUDIO_Default_308.mp3', sr=44100)
print(orig_sr)
resampled_ori = librosa.core.resample(ori_mp3, orig_sr=orig_sr, target_sr=8000)
sd.play(resampled_ori, samplerate=8000)
sd.wait()
mu = mel_file_to_audio('mel/0.npy', ori_sr=8000, mel_sr=8000)
print(mu.shape)
sd.play(mu, samplerate=8000)

start
44100
(40960,)
